# Object oriented error handling
- Errors are defined by classes
- an error object is instantiated at error time
    - the class used tells you something about the problem
    - the error object may include specific information about the problem, like a file path that doesn't exist
- errors can be raised/thrown either by your code, or Python itself
- errors may be caught/handeled by your code
- Java - compilier DEMANDS you handle all kinds of potential errors
- Python - hey whatever...
- but, if you get an error, and there's no handler, you CRASH
- [List of builtin errors]
(https://docs.python.org/3/library/exceptions.html#exception-hierarchy)
- error types form a hierarchy(formed by using single inheritance)


# can explicitly 'raise'(or throw) an error 
- 'raise' statement takes an error object, and invokes the python error handling system on it

In [1]:
def picky(n):
    if n == 3:
        raise ValueError('picky is unhappy - bad value n={}'.format(3)) # similar to throw in Java
    return n+1

# no problem here
picky(1)

2

In [2]:
# ValueError will be raised/thrown
# BAD ENDING

picky(3)

ValueError: picky is unhappy - bad value n=3

# catch/handle errors with try/except

In [3]:
try:
    picky(3)
except Exception as e:
    # e is the error object from the raise statement
    print(e)

'now we are here' 

picky is unhappy - bad value n=3


'now we are here'

In [6]:
# read a number - insist!

def getn():
    while True:
        try:
            s = input('gimme a number!!')
            j = int(s)
            return j
        except Exception:
            print('Please give me a number')

getn()
        

gimme a number!!32dsaf
Please give me a number
gimme a number!!323


323

# 'finally' clause is always executed, with or without error
- gives you a chance to do something before
you lose control

In [7]:
try:
    picky(1)
except Exception:
    print('got error')
finally:
    print('finally')
'now we are here'

finally


'now we are here'

In [9]:
try:
    picky(3)
except Exception:
    print('got error')
finally:
    print('finally') # for cleanup after a computation (cleanign up the mess whether it failed or not)
'now we are here'

got error
finally


'now we are here'

# call stack frames display
- if no error handler is found, the top
level will display a 'stack trace', and your entire computation is terminated
- shows context of error
- very useful for debugging
- note: the most recent call frame is printed last

In [10]:
def A(n):
    B(1)

def B(n):
    C(2)

def C(n):
    D(3)
    
def D(n):
    raise Exception("stack frames will be displayed")

A(10)

Exception: stack frames will be displayed

# can define a custom error class
- usually inherit from 'Exception'

# breaking out of nested loops                    

In [11]:
# note use of 'pass'

class breakNested(Exception):
    pass

def bn(bval):
    try:
        for x in range(3):
            print('x',x)
            for y in range(3):
                print('y',y)
                if y == bval:
                    raise breakNested
    except breakNested:
        print('broke out of nested loop')
                       


In [12]:
bn(9)

x 0
y 0
y 1
y 2
x 1
y 0
y 1
y 2
x 2
y 0
y 1
y 2


In [13]:
bn(2)

x 0
y 0
y 1
y 2
broke out of nested loop


In [14]:
class missedSecret(Exception):
    def __init__(self, secret, guess):
        self.secret = secret
        self.guess = guess
    
    def __str__(self):
        return 'secret was {}, guess was {}'\
    .format(self.secret, self.guess)
    
def guessSecret(guess):
    secret = 1234
    if guess != secret:
        raise missedSecret(secret, guess)
    return 'guess is correct'
    
guessSecret(1234)

'guess is correct'

In [15]:
guessSecret(34)

missedSecret: secret was 1234, guess was 34

# can get error object and examine it
- info in error object might help decide how to handle the error

In [16]:
try:
    guessSecret(12)
except missedSecret as ms:
    # ms will be bound to the error object
    print(ms.secret, ms.guess)

1234 12


# when an error is raised, Python will search the call stack for an error handler
- Python checks the current stack frame for a handler, then checks each
older frame in turn
- if no handler is found, the error is printed by the top level(and your program dies)

In [17]:
# no error handler in 'bar', so look in caller, 'foo',
# which does have one.

def foo():
    try:
        bar()
    except Exception as e:
        print('caught in foo:', e)
    
def bar():
    a,b = 0,1
    # division by zero error raised here
    b/a

foo()

caught in foo: division by zero


In [18]:
# 'bar' has a handler, 
# so error is caught there

def foo():
    try:
        bar()
    except Exception as e:
        print('caught in foo:', e)
    
def bar():
    a,b = 0,1
    try:
        b/a
    except Exception as e:
        print('caught in bar:', e)

foo()

caught in bar: division by zero


In [19]:
# both 'foo' and 'bar' have handlers, but
# neither is the right type, so the error
# is NOT caught

def foo():
    try:
        bar()
    except FileNotFoundError as e:
        print('caught in foo:', e)
    
def bar():
    a,b = 0,1
    try:
        b/a
    except ValueError as e:
        print('caught in bar', e)

foo()

ZeroDivisionError: division by zero

# Complex error example
- you can ignore this if you wish

In [20]:
def tc(b, f):
    if b:
        try:
            return tc2(f)
        # usually a good idea to catch
        # Exception at top level
        except Exception as e:
            print('tc: caught {}'.format(e))
    else:
        try:
            return tc2(f)
        except OSError as os:
            print('tc: caught {}'.format(os))  

def tc2(f):
    try:
        return tc3(f)
    # can catch any number of error types
    # in a single try
    except FileNotFoundError as fe:
        print('tc2: caught {}'.format(fe))
    except MemoryError as me:
        print('tc2: caught {}'.format(me))
    
def tc3(f):
    try:
        return f()
    except ArithmeticError as ae:
        print('tc3: caught {}'.format(ae))  
    
def noproblem():
    a = 5/4
    return a
    
def dbz2():
    try: 
        c = 1/0
        return c
    except ZeroDivisionError as z:
        print('dbz: caught {}'.format(z))
        
def dbz3():
    a = 1/0
    return a
    
def si():
    raise StopIteration
    
def fnf():
    # can put useful information about 
    # the error into the error object
    raise FileNotFoundError('/tmp/foo')

def me():
    raise MemoryError

In [21]:
# should run fine, and not generate an error

tc(1, noproblem)

1.25

In [22]:
#  caught the error it generated, inside dbz2

tc(1, dbz2)

dbz: caught division by zero


In [23]:
# no handler in dbz3
# caught by handler in tc3

tc(1, dbz3)

tc3: caught division by zero


In [24]:
# caught in 'tc' Exception handler

tc(1, si)

tc: caught 


In [25]:
# no handler on the call stack
# top level prints stack trace, 
# and your program crashes

tc(0, si)

StopIteration: 